In [39]:
import numpy as np
from math import pow, sqrt

yr_to_sec = 31557600.0

flood_discharge = 750.
reach_length = 30000.
transition__init = 150000. # sgsl
base_level__init = 5.
base_level__rate = 6 / yr_to_sec / 1000. # mm/yr # ksidot
subsidence__rate = 0. / yr_to_sec / 1000. # mm/yr
sea_level_rise__start_year = 0 / yr_to_sec
sea_level_rise__end_year = 12000 / yr_to_sec
dt = 0.2 * yr_to_sec
total_time = 600. * yr_to_sec
print_dt = 100. * yr_to_sec

reach_length__sand = reach_length - transition__init

g = 9.81

rho_w = 1000.
rho_s = 2650.

R = (float(rho_s) - float(rho_w)) / float(rho_w)

##################################################

sed_discharge__sand = 0.15
flood_intermittency__sand = 0.05
grain_size__sand = 0.25 / 1000.
Chezy_coeff__sand = 25.
channel_width__sand = 90.
slope__sand = 0.00015
floodplain_width__sand = 4000.
sinuosity__sand = 2.
fraction_deposited__sand = 1.
porosity__sand = 0.4
num_nodes__sand = 30.

dxbar_sand = 1. / num_nodes__sand
Cf_sand = 1. / pow(Chezy_coeff__sand, 2)
qw_sand = flood_discharge / channel_width__sand
qf_sand = sed_discharge__sand / channel_width__sand
channel_fraction__sand = floodplain_width__sand / channel_width__sand # rBg
If_eff__sand = (flood_intermittency__sand * (1. + fraction_deposited__sand) *
                sinuosity__sand / channel_fraction__sand)


##################################################

sed_discharge__gravel = 0.075
flood_intermittency__gravel = 0.05
grain_size__gravel = 30. / 1000.
Chezy_coeff__gravel = 15.
channel_width__gravel = 90.
slope__gravel = 0.0015
floodplain_width__gravel = 4000.
sinuosity__gravel = 1.5
fraction_deposited__gravel = 0.5
porosity__gravel = 0.4
num_nodes__gravel = 30.

dxbar_gravel = 1. / num_nodes__gravel
Cf_gravel = 1. / pow(Chezy_coeff__gravel, 2)
qw_gravel = flood_discharge / channel_width__gravel
qf_gravel= sed_discharge__gravel / channel_width__gravel
channel_fraction__gravel = floodplain_width__gravel / channel_width__gravel # rBs
If_eff__gravel = (flood_intermittency__gravel * (1. + fraction_deposited__gravel) *
                sinuosity__gravel / channel_fraction__gravel)

##################################################
# CHECKS
##################################################

if base_level__rate > subsidence__rate:    
    Lmax = ((1. / base_level__rate) *
           ((If_eff__gravel * qf_gravel / (1 - porosity__gravel)) +
            If_eff__sand * (qf_sand - (fraction_deposited__gravel * qf_gravel)) / (1 - porosity__sand)))  
else:    
    Lmax = ((1. / subsidence__rate) *
           ((If_eff__gravel * qf_gravel / (1 - porosity__gravel)) +
            If_eff__sand * (qf_sand - (fraction_deposited__gravel * qf_gravel)) / (1 - porosity__sand)))
                                       
assert reach_length < Lmax, 'Reach length musch be smaller than Lmax value!'

###

qf_star__sand = qf_sand / (sqrt(R * g * grain_size__sand) * grain_size__sand)

tau__sand = pow(Cf_sand * qf_star__sand / 0.05, 0.4)

Ss = (pow(R * grain_size__sand * tau__sand, 1.5) / 
      (sqrt(Cf_sand * pow(qw_sand, 2) * g)))
Hs = pow(Cf_sand * pow(qw_sand, 2) / (g * Ss) , 1./3)
Froude_sand = sqrt(pow(qw_sand, 2) / (g * pow(Hs, 3)))

assert Froude_sand < 1, 'Froude number for sand must be less than 1'

###

tau__gravel = 0.035
qf_star__gravel = qf_gravel / (sqrt(R * g * grain_size__gravel) * grain_size__gravel)

f = 1.
while abs(f) > 1e-10:
    
    f = (11.2 * pow(tau__gravel, 1.5) *
         pow(1. - (0.03 / tau__gravel), 4.5) - qf_star__gravel)
    
    fprime = (1.5 * 11.2  * sqrt(tau__gravel) *
              pow(1. - (0.03 / tau__gravel), 4.5) +
              11.2 * 4.5 * pow(1. - (0.03 / tau__gravel), 3.5) *
              0.03 / sqrt(tau__gravel))
    
    tau__gravel += - f / fprime
    
Sg = (pow(R * grain_size__gravel * tau__gravel, 1.5) / 
      (sqrt(Cf_gravel * pow(qw_gravel, 2) * g)))
Hg = pow(Cf_gravel * pow(qw_gravel, 2) / (g * Sg) , 1./3)

Froude_gravel = sqrt(pow(qw_gravel, 2) / (g * pow(Hg, 3)))

assert Froude_gravel < 1, 'Froude number for gravel must be less than 1'

###

base_level__min = pow(pow(qf_sand,2) / g, 1./3)

assert base_level__init > base_level__min, 'Downstream water surface must be higher than the minimum'




In [38]:
base_level__init

5.0

In [23]:
((If_eff__gravel * qf_gravel / (1 - porosity__gravel)))

3.5156250000000008e-06

In [25]:
If_eff__sand * (qf_sand - (fraction_deposited__gravel * qf_gravel)) / (1 - porosity__sand)

9.375e-06

In [26]:
(1. / base_level__rate)

5.281347969004825e-06